In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import plastic_card_pose
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
from geometry_msgs.msg import WrenchStamped
import time
rospy.init_node('scoop', anonymous=True) #initialize the node



In [2]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1612186933.084637]: Client test: Starting sending goals


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [4]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass

In [5]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    while True:
        rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
        camPstone = np.array([pose_x, pose_y, pose_z, 1])
        eeTcam = m3d.Transform()
        eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
        eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
        eeTcam.orient.rotate_xb(eeTcam_e[0])
        eeTcam.orient.rotate_yb(eeTcam_e[1])
        eeTcam.orient.rotate_zb(eeTcam_e[2])
        tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
        eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
        tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
        worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
        worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
        if worldTstone[2]<0:
            continue
        else:
            return [worldTstone[0]+0.02-0.0006+0.002, worldTstone[1]+0.02148-0.002+0.006-0.005-0.003, worldTstone[2]]

In [7]:
import json
def hori_stone_pose_callback(msg):
    global pose_Go_stone_set, Go_stone_number, yaw_pitch_normal_set, hori_box_set, hori_json_error
    Go_stone_number = int(msg.Go_stone_number)
    if Go_stone_number == 0:
        pose_Go_stone_set = []
        yaw_pitch_normal_set = []
        hori_box_set = []
    else:
        #try:
        if True:
            pose_x = msg.x
            pose_y = msg.y
            pose_z = msg.z
            yaw = msg.yaw
            pitch = msg.pitch
            normal = msg.normal
            pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
            yaw_pitch_normal_set.append([yaw,pitch,normal])
            box = json.loads(msg.box)
            box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
            hori_box_set.append(box)
        #except:
            #hori_json_error = True
hori_pose_topic = '/plastic_card_pose'
rospy.Subscriber(hori_pose_topic, plastic_card_pose, hori_stone_pose_callback)

In [8]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img, depth_array
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/plastic_card/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/plastic_card/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/plastic_card/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [9]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [10]:
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_finger_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    return traslating_wst_world, extension_distance

def pos_flength_adjust_thumb_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_r):
    theta=theta*pi/180
    current_flength_l = flength_r - (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r)  
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, current_flength_l, flength_r) 
    traslating_wst_g = [current_right_fingertip_pos_g[0]-next_right_fingertip_pos_g[0], current_right_fingertip_pos_g[1]-next_right_fingertip_pos_g[1]]
    traslating_wst_world = np.array([[sin(theta), -cos(theta)], [cos(theta), sin(theta)]]).dot(np.array([[traslating_wst_g[0]], [traslating_wst_g[1]]])).ravel().tolist()
    next_left_fingertip_pos_g = [next_left_fingertip_pos_g[0]+traslating_wst_g[0], next_left_fingertip_pos_g[1]+traslating_wst_g[1]]
    next_right_fingertip_pos_g = [next_right_fingertip_pos_g[0]+traslating_wst_g[0], next_right_fingertip_pos_g[1]+traslating_wst_g[1]]
    extension_distance = -(next_right_fingertip_pos_g[1]+(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r)/tan(theta)-next_left_fingertip_pos_g[1])
    traslating_wst_world[0]=traslating_wst_world[0]-((2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)-(2*l0+2*l1*sin(next_alpha*pi/180)-l2l-l2r))/sin(theta)
    return traslating_wst_world, extension_distance

In [14]:
def skim_grasp_execute(Go_index):
    print 'Go_index', index
    global pose_Go_stone_set, yaw_pitch_normal_set, hori_box_set, depth_array
    pose_x = pose_Go_stone_set[Go_index][0]
    pose_y = pose_Go_stone_set[Go_index][1]
    pose_z = pose_Go_stone_set[Go_index][2]
    normal=yaw_pitch_normal_set[Go_index][2]
    ####################### change tcp pose ############################
    rob.set_tcp((0.028, 0, 0.338, 0, 0, 0))  
    rospy.sleep(0.5)
    ####################################################################
    go_to_safe_place()    
    worldPstone = np.array([pose_x, pose_y, pose_z, 1])
    #print 'worldPstone', worldPstone
    tcpTstone = np.transpose(np.matmul(np.linalg.inv(rob.get_pose().get_matrix()), worldPstone))
    #print "tcpTstone", tcpTstone
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    #print eeTcam
    
    yaxis = (0, 1, 0)
    zaxis = (0, 0, 1)
    Ry = tf.transformations.rotation_matrix(pi/2, yaxis)
    Rz = tf.transformations.rotation_matrix(-pi/2, zaxis)
    eeTtcp = np.matmul(Ry, Rz)
    ####################### change tcp pose ############################
    eeTtcp[:3,3] = np.array([0.338,-0.028,0])
    ####################################################################
    tcpTee = inv(eeTtcp)
    #print tcpTee

    n_normalize = normal / np.linalg.norm(normal)
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    eeTnormal = np.matmul(np.array([[0, 0, 1, 0],[0, 1, 0, 0], [-1,0,0,0], [0,0,0,1]]), n_normalize)
    tcpTnormal = np.matmul(tcpTee[0:3,0:3], eeTnormal[0:3])
    worldTnormal = [tcpTnormal[1], tcpTnormal[0], tcpTnormal[2]]

    box = hori_box_set[Go_index]
    length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
    length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
    if length1 < length2:
        midpoint0 = [(box[1][0]+box[2][0])/2, (box[1][1]+box[2][1])/2]
        midpoint1 = [(box[0][0]+box[3][0])/2, (box[0][1]+box[3][1])/2]
    else:
        midpoint0 = [(box[0][0]+box[1][0])/2, (box[0][1]+box[1][1])/2]
        midpoint1 = [(box[2][0]+box[3][0])/2, (box[2][1]+box[3][1])/2]
    center = [(midpoint0[0]+midpoint1[0])/2, (midpoint0[1]+midpoint1[1])/2]
    dir0 = [midpoint0[0]-center[0], midpoint0[1]-center[1], 0]
    dir1 = [midpoint1[0]-center[0], midpoint1[1]-center[1], 0] 
    if np.dot(dir0, worldTnormal)>0:
        high_to_low_dir = dir0
    else:
        high_to_low_dir = dir1
    rotate_angle = atan2(high_to_low_dir[1], high_to_low_dir[0])-atan2(1,0)
    if rotate_angle > 0:
        rotate_angle = rotate_angle % (2*pi)
    else:
        rotate_angle = rotate_angle % (-2*pi)
    rotate_angle0 = -rotate_angle
    if abs(rotate_angle0)>120*pi/180:
        if rotate_angle0>0:
            rotate_angle0-=pi
        else:
            rotate_angle0+=pi
    if str(rotate_angle0) == 'nan':
        return False
    psi = 30*pi/180
    
    Robotiq.goto(robotiq_client, pos=12*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  # need change
    rospy.sleep(0.5)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    rob.movel_tool((tcpTstone[0,0]+0.002, tcpTstone[1,0]-0.004, 0, 0, 0, 0), acc=0.3, vel=1, wait=True)
    rob.translate_tool((0, 0, tcpTstone[2,0]-0.015), acc=0.5, vel=0.8)
    rob.movel_tool((0,0,0,0,0,rotate_angle0), acc=0.5, vel=0.8, wait=True)
    if rotate_angle0>0:
        rob.translate_tool((0.020, 0, 0), acc=0.06, vel=0.08)   # need change
    else:
        rob.translate_tool((0.017, 0, 0), acc=0.06, vel=0.08)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    rob.translate_tool((-0.02*sin(psi), 0, 0.02*cos(psi)), acc=0.01, vel=0.05, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.5:   
            num_large_force += 1
        if time.time()-time0>2.8:  
            break
    time.sleep(0.5)
    '''
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length_extend in np.arange(0,50,4):
        current_finger_length = 0+current_finger_length_extend #Change
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.5:
            break
    '''
    current_fingerlength_int=0
    print current_fingerlength_int  
    angle_step=0.2
    for angle in np.arange(12,-4,-angle_step):
        translate_dir_dis, extension_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    #go_to_end_place()
    #rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=2*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)
    #rospy.sleep(1)
    #ser.write('1000')
    #rospy.sleep(1)
    #rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True) 
    return True


In [15]:
executed_time_man_array = []
executed_time_vis_array = []

stone_img_count = 0
hori_json_error = False
while True:
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    Go_stone_number = None
    pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    hori_box_set = []
    is_send_stone_img = 1
    time_count_vis_0 = time.time()
    print 'started', stone_img_count
    while True:
        if Go_stone_number != None and Go_stone_number == len(hori_box_set):
            break  
        elif Go_stone_number != None and hori_json_error==True:
            rospy.sleep(0.5)
            break   
        rospy.sleep(0.1)
    vis_time = time.time()-time_count_vis_0
    executed_time_vis_array.append(vis_time)
    print 'vis_time', vis_time
    print pose_Go_stone_set
    print yaw_pitch_normal_set
    time_count_man_0 = time.time()
    priority_list = []
    for index_hori in range(len(pose_Go_stone_set)):
        priority_list.append(('h', index_hori))
    print priority_list
    for index in priority_list:
        if index[0]=='h':
            if pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if skim_grasp_execute(index[1])==True:
                man_time = time.time()-time_count_man_0
                executed_time_man_array.append(man_time)
                print 'man_time', man_time
                break
    else:
        while True:
            if_human_shuffle_finished = raw_input("Human shuffle. Finished? (y or n)")
            if if_human_shuffle_finished == 'y':
                break

continue?1
started 0
dsag
0
vis_time 3.20638298988
[[0.05425162916477402, 0.682255908038385, 0.023226097992319616]]
[[2.590576784736477, 0.21406068356382169, (0.011168548605026743, 0.043290815628629076, -0.9058777784966553)]]
[('h', 0)]
Go_index ('h', 0)
0
man_time 41.0741660595
continue?1
started 1
dsag
1
vis_time 3.60543608665
[[0.041867716158482116, 0.6584610422894868, 0.02305525687514981]]
[[-2.7534315895229104, 0.21406068356382169, (-0.011117298158871173, 0.0516872359294717, -0.9417591381879095)]]
[('h', 0)]
Go_index ('h', 0)
0
man_time 46.0125379562
continue?1
started 2
dsag
2
vis_time 4.3100039959
[[0.0667498453129194, 0.6981864118001964, 0.025431755928978894], [0.012225885382643709, 0.6754467989950987, 0.026818229355110412]]
[[2.868898317044126, 0.6365082157879507, (0.06142869736228838, 0.03626312645144674, -0.9644679155716946)], [3.010765257515736, 0.4101273405414903, (0.012192246301559085, 0.05213561555182771, -0.9533366916040048)]]
[('h', 0), ('h', 1)]
Go_index ('h', 0)
0
ma

KeyboardInterrupt: 

In [20]:
((0.075-0.07441812768708338)+(0.05-0.04897243287305203)+(0.025-0.023395635598574306)+(0.025-0.0254619238759079))/4.0

0.0006879699913455977

In [21]:
((0.675-0.6798453558741636)+(0.675-0.679548104224364)+(0.7-0.7027561247441642)+(0.65-0.6524424649435531))/4.0

-0.003648012446561183

In [15]:
go_to_home()

In [7]:
softtip()

In [18]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


In [19]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [4]:
from scipy.spatial.transform import Rotation as R
import numpy as np
r = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)])
r.as_matrix()

AttributeError: 'Rotation' object has no attribute 'as_matrix'

In [7]:
import time
time0 = time.time()
rob.translate((0,0,-0.035), acc=0.001, vel=0.01, wait=True)
print time.time()-time0

12.1238350868


In [18]:
eeTnormal

NameError: name 'eeTnormal' is not defined

In [23]:
eeTcam.get_matrix()

NameError: name 'eeTcam' is not defined

In [18]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)

In [23]:
    current_fingerlength_int = 0
    angle_step = 0.2
    for angle in np.arange(12,-4,-angle_step):
        translate_dir_dis, extension_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.019, 0.01, 0.125)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int+extension_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)

In [16]:
np.mean(executed_time_man_array)

42.502858440081276

In [17]:
np.mean(executed_time_vis_array)

4.196339991357592